70019.90s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 1.1 MB/s eta 0:00:0000:0100:010m


In [2]:
from langchain_community.llms import Ollama
import pandas as pd
import glob
import os
import yaml
from rapidfuzz import process  # modern replacement for fuzzywuzzy

model = "gemma3"  # Specify your model here
# Load the YAML file
with open("data/templates/GHG_templates.yaml", "r") as f:
    templates = yaml.safe_load(f)

data_folder = "data/sample_GHG_data/"
data_files = glob.glob(os.path.join(data_folder, "*.xlsx"))

GHG_categories = list(templates.keys())  # Assuming YAML file has GHG categories as keys
print("GHG Categories:", GHG_categories)

def match_category(input_str, choices, threshold=80):
    """
    Match input string to the closest category.
    Returns best match if similarity >= threshold, else None.
    """
    best_match, score, _ = process.extractOne(input_str, choices)
    return best_match if score >= threshold else None

# Initialize LLM once
ollama = Ollama(
    base_url="http://localhost:11434",
    model=model
)

results = []

for file in data_files[:]:
    df = pd.read_excel(file)

    # Extract ground truth
    if 'GHGCategory' in df.columns:
        gt_value = str(df['GHGCategory'].iloc[0])
    elif 'GHG Category' in df.columns:
        gt_value = str(df['GHG Category'].iloc[0])
    else:
        gt_value = 'Not GHG-related'

    # Prepare the prompt with sample data
    sample_data = df.head(3).to_string(index=False)

    prompt = f"""
    Here are the first 3 rows of an Excel file:

    {sample_data}

    Analyze this sample data and classify it into one of the following GHG categories:

    {GHG_categories}

    Important rules:
    - Respond with the exact category name from the list above.
    - If none of the categories apply, respond with "Not GHG-related".
    - Do not provide explanations, just the category name.
    """

    # Call LLM
    response = ollama.invoke(prompt).strip()
    print(f"{file} → {response} | Ground Truth: {gt_value}")

    # Compare response with ground truth
    matched = match_category(input_str=response, choices=[gt_value])
    process_template = 'Yes' if matched else 'No'

    # Collect result row
    results.append({
        "Filename": os.path.basename(file),
        "Response": response,
        "Ground Truth": gt_value,
        "process_template": process_template
    })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Save to Excel
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_folder = "output_data"
os.makedirs(output_folder, exist_ok=True)
output_excel = os.path.join(output_folder, f"classify_{timestamp}.xlsx")
results_df.to_excel(output_excel, index=False)

print(f"Summary saved to {output_excel}")


GHG Categories: ['Scope_1_Fugitives', 'Scope_2_PurchasedEnergy', 'SScope_3_1_PurchasedGoods&Services_SpendBasedMethod']
data/sample_GHG_data/Scope_1_Direct_Emissions_Fugitives.xlsx → Scope_1_Fugitives | Ground Truth: Scope_1_Fugitives
data/sample_GHG_data/Scope_3_1_Purchased_Goods_and_Services_Spend_Based_Method.xlsx → SScope_3_1_PurchasedGoods&Services_SpendBasedMethod | Ground Truth: Scope_3.1_PurchasedGoods&Services_SpendBasedMethod
data/sample_GHG_data/Scope_2_Indirect_Emissions_Purchased_Energy.xlsx → Scope_2_PurchasedEnergy | Ground Truth: Scope_2_PurchasedEnergy
Summary saved to output_data/classify_20250821_112757.xlsx


# calculator payload mapping

In [7]:
import os
import json
import yaml
import pandas as pd
import ast
from langchain_community.llms import Ollama

model = "gemma3"  # Specify your model here
# Load results Excel
# Load YAML once
with open("data/templates/GHG_templates.yaml", "r") as f:
    templates = yaml.safe_load(f)

data_folder = "data/sample_GHG_data/"
ollama = Ollama(base_url="http://localhost:11434", model=model)

output_template_payload = {}
classify_out_df = pd.read_excel(output_excel)
for idx, row in classify_out_df.iterrows():
    if row["process_template"] == "Yes":
        ground_truth = row["Ground Truth"]
        filename = row["Filename"]

        payload_template = templates.get(ground_truth, {})
        scope_data = pd.read_excel(os.path.join(data_folder, filename))
        n_rows = len(scope_data)

        # Convert dataframe rows to JSON-style records
        sample_data = scope_data.to_dict(orient="records")

        prompt = f"""
        You are a data processing assistant. 

        Below is the payload template for each row:  

        {payload_template}

        Here are all {n_rows} rows from the Excel file as JSON records:
        {sample_data}

        For each row, generate **one dictionary** that fills in the template 
        using values from that row.  

        Return a  list of {n_rows} dictionaries**, one per row.  
        Do not add explanations. Do not wrap in code fences.  
        """

        # Get response
        response = ollama.invoke(prompt).strip()

        # Clean up if model still adds code fences
        if response.startswith("```"):
            response = response.strip("`")
            response = response.split("\n", 1)[-1]
            response = response.rsplit("\n", 1)[0]

        # Parse into Python list
        try:
            try:
                parsed = ast.literal_eval(response)
            except Exception:
                parsed = json.loads(response)  # fallback if it's JSON
        except Exception as e:
            print(f"⚠️ Failed to parse response for {filename}: {e}")
            parsed = response

        output_template_payload[filename] = parsed
        print(f"✅ Generated {len(parsed) if isinstance(parsed, list) else 'unknown'} payloads for {filename}")

# Save to JSON
import json

output_folder = "output_data"
os.makedirs(output_folder, exist_ok=True)
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_json = os.path.join(output_folder, f"template_payload_{timestamp}.json")
with open(output_json, "w") as f:
    json.dump(output_template_payload, f, indent=4)

print(f"🎯 Saved generated payloads to {output_json}")


⚠️ Failed to parse response for Scope_1_Direct_Emissions_Fugitives.xlsx: Expecting property name enclosed in double quotes: line 3 column 5 (char 10)
✅ Generated unknown payloads for Scope_1_Direct_Emissions_Fugitives.xlsx
✅ Generated 3 payloads for Scope_3_1_Purchased_Goods_and_Services_Spend_Based_Method.xlsx
✅ Generated 5 payloads for Scope_2_Indirect_Emissions_Purchased_Energy.xlsx
🎯 Saved generated payloads to output_data/template_payload_20250821_124014.json


{'Scope_1_Direct_Emissions_Fugitives.xlsx': [{'Facility Name': 'testmulti',
   'Refrigerant': 'HFC-236cb (R-236cb)',
   'Quantity of refrigerant leakage': 47.0,
   'Unit(s)': 'gram (g)',
   'Refrigerant inventory (in storage, not equipment) at the beginning of the year': 2.0,
   'Refrigerant inventory (in storage, not equipment) at the end of the year': 3.0,
   'Refrigerant purchased from producers/distributors': 4.0,
   'Refrigerant returned by equipment users': 5.0,
   'Refrigerant returned after off-site recycling or reclamation': 6.0},
  {'Facility Name': 'testmulti',
   'Refrigerant': 'HFC-236cb (R-236cb)',
   'Quantity of refrigerant leakage': 48.0,
   'Unit(s)': 'gram (g)',
   'Refrigerant inventory (in storage, not equipment) at the beginning of the year': 2.0,
   'Refrigerant inventory (in storage, not equipment) at the end of the year': 3.0,
   'Refrigerant purchased from producers/distributors': 4.0,
   'Refrigerant returned by equipment users': 5.0,
   'Refrigerant returned

In [6]:

output_folder = "output_data"
os.makedirs(output_folder, exist_ok=True)
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_json = os.path.join(output_folder, f"template_payload_{timestamp}.json")
with open(output_json, "w") as f:
    json.dump(output_template_payload, f, indent=4)

print(f"🎯 Saved generated payloads to {output_json}")


🎯 Saved generated payloads to output_data/template_payload_20250821_114901.json
